# Aplicando Transformers en Reconocimiento de Personalidad.

**Investigadores**: <br>
  Dr. Ramón Zatarain Cabada<br>
  Dra. María Lucía Barrón Estrada<br>
  M.C. Víctor Manuel Bátiz Beltrán

# PersonText

En este cuaderno se muestra el uso de Transformers con el corpus PersonText.

**Referencias**:

- Bátiz Beltrán, V. M., Zatarain Cabada, R., & Barrón Estrada, M. L. (2024). Creation of a Corpus in Spanish for the recognition of personality traits. Computación y Sistemas, Volumen 28, No. 3, 2024. ISSN: 2007-9737. Artículo disponible en línea: https://cys.cic.ipn.mx/ojs/index.php/CyS/article/view/4619/3734

In [1]:
!pip install emoji

In [2]:
import re
#import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import SnowballStemmer
import unicodedata
from collections import Counter
from wordcloud import WordCloud
from gensim.utils import simple_preprocess
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
#import seaborn as sns
#from sklearn.metrics import confusion_matrix
#import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import emoji
import keras
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
print('Listo')

Listo


#Procedimiento

## 1.Carga del conjunto de Datos

## Descarga del Corpus

In [3]:
def corpus_download(path, url):
  !wget --no-check-certificate \
     {url} \
     -O {path}

In [4]:
corpus_download("persontext.csv","https://catalabs.mx/datasets/persontext/corpus_persontext_v1.csv")
corpus_download("persontextV2.csv","https://catalabs.mx/datasets/persontext/corpus_persontext_v2.csv")
corpus_download("mypersonality_persontext.csv","https://catalabs.mx/datasets/persontext/corpus_mypersonality_esp.csv")

--2024-10-22 08:09:11--  https://catalabs.mx/datasets/persontext/corpus_persontext_v1.csv
Resolving catalabs.mx (catalabs.mx)... 174.136.30.149
Connecting to catalabs.mx (catalabs.mx)|174.136.30.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 159288 (156K) [text/csv]
Saving to: ‘persontext.csv’

persontext.csv      100%[===================>] 155.55K  --.-KB/s    in 0.1s    

2024-10-22 08:09:11 (1.19 MB/s) - ‘persontext.csv’ saved [159288/159288]

--2024-10-22 08:09:11--  https://catalabs.mx/datasets/persontext/corpus_persontext_v2.csv
Resolving catalabs.mx (catalabs.mx)... 174.136.30.149
Connecting to catalabs.mx (catalabs.mx)|174.136.30.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 189361 (185K) [text/csv]
Saving to: ‘persontextV2.csv’

persontextV2.csv    100%[===================>] 184.92K  --.-KB/s    in 0.1s    

2024-10-22 08:09:12 (1.44 MB/s) - ‘persontextV2.csv’ saved [189361/189361]

--2024-10-22 08:09:12--  https:/

## Selección del Corpus a utilizar.

In [5]:
corpus_dict = {
    'PersonText' : 'persontext.csv',
    'PersonTextV2' : 'persontextV2.csv',
    'MyPersonalityPT' : 'mypersonality_persontext.csv'
}

corpus_seleccionado = 'PersonText' #@param ['PersonText','PersonTextV2','MyPersonalityPT']
corpus = pd.read_csv(corpus_dict[corpus_seleccionado])


## Exploración del Corpus

In [6]:
corpus.tail()

,uid,texto,fecha_prueba,apertura,presenta_apertura,responsabilidad,presenta_responsabilidad,sociabilidad,presenta_sociabilidad,amabilidad,presenta_amabilidad,neuroticismo,presenta_neuroticismo,fechaNacimiento,sexo,edad_calculada,Experimento
208,DOIxrvGJKHexpjsBAPh4zf5eXyQ2,Bueno chavos vamos a ponernos pilas con el tr...,01/11/2021,0.5,No,0.8,Sí,0.4,No,0.8,Sí,0.3,No,06/02/1999,Masculino,22,Exp 3-1
209,B6AJrl8ECWe2rIUCiO5TuYxeLwl1,Bueno pues considero yo que para mi persona e...,04/03/2021,0.7,Sí,0.8,Sí,0.7,Sí,0.5,No,0.9,Sí,02/12/1995,Masculino,25,Exp 1-2
210,iXWwNMyEN5OBn4F6nMQnoqxNoki2,Hola el video de hoy se tratará más que nada ...,04/03/2021,0.8,Sí,0.8,Sí,0.3,No,0.8,Sí,0.9,Sí,23/07/1994,Masculino,26,Exp 1-3
211,n3a5RSlooCZYAuoefM4gCirmh9I2,Son cinco las características que debe tener ...,04/03/2021,0.5,No,0.5,No,0.6,No,0.6,No,0.7,Sí,24/10/1995,Masculino,25,Exp 1-3
212,m8zpZe9PXmQ9tRLjB7pVSCZwoYv2,Bueno pues un tema que me interesa a mí son l...,26/03/2021,0.9,Sí,0.7,Sí,0.2,No,0.7,Sí,0.6,No,10/07/1997,Masculino,23,Exp 1-1


Dejamos solo las columnas con los datos que vamos a trabajar.

In [7]:
#                    0                     1                       2                      3                       4
rasgos = ['presenta_apertura','presenta_responsabilidad','presenta_sociabilidad','presenta_amabilidad','presenta_neuroticismo']

#rasgos = ['presenta_apertura', 'presenta_neuroticismo']

rasgo = rasgos[0]

corpus_limpio = corpus[['texto',rasgo]]


Visualizamos el Corpus resultante.

In [8]:
corpus_limpio.tail()

,texto,presenta_apertura
208,Bueno chavos vamos a ponernos pilas con el tr...,No
209,Bueno pues considero yo que para mi persona e...,Sí
210,Hola el video de hoy se tratará más que nada ...,Sí
211,Son cinco las características que debe tener ...,No
212,Bueno pues un tema que me interesa a mí son l...,Sí


Reemplazamos las etiquetas "Sí" y "No" por "1" y "0" respectivamente.

In [10]:
replacement_mapping = {'Sí': 1, 'No': 0, 'y': 1, 'n':0}
corpus_limpio[rasgo] = corpus_limpio[rasgo].replace(replacement_mapping)

In [21]:
corpus_limpio.tail()

,texto,presenta_apertura
208,Bueno chavos vamos a ponernos pilas con el tr...,0
209,Bueno pues considero yo que para mi persona e...,1
210,Hola el video de hoy se tratará más que nada ...,1
211,Son cinco las características que debe tener ...,0
212,Bueno pues un tema que me interesa a mí son l...,1


## 2. Limpieza de los Datos

In [11]:
def process_text(sentence, norm_user = True, norm_hashtag = True, separate_characters = True):
    # Convert instance to string
    sentence = str(sentence)

    # All text to lowecase
    sentence = sentence.lower()

    # Normalize users and url
    if norm_user == True:
        sentence = re.sub(r'\@\w+','@usuario', sentence)
    if norm_hashtag == True:
        sentence = re.sub(r"http\S+|www\S+|https\S+", 'url', sentence, flags=re.MULTILINE)

    # Separate special characters
    if separate_characters == True:
        sentence = re.sub(r":", " : ", sentence)
        sentence = re.sub(r",", " , ", sentence)
        sentence = re.sub(r"\.", " . ", sentence)
        sentence = re.sub(r"!", " ! ", sentence)
        sentence = re.sub(r"¡", " ¡ ", sentence)
        sentence = re.sub(r"“", " “ ", sentence)
        sentence = re.sub(r"'", " ' ", sentence)
        sentence = re.sub(r"”", " ” ", sentence)
        sentence = re.sub(r"\(", " ( ", sentence)
        sentence = re.sub(r"\)", " ) ", sentence)
        sentence = re.sub(r"\?", " ? ", sentence)
        sentence = re.sub(r"\¿", " ¿ ", sentence)

    # Substituting multiple spaces with single space
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)
    # emojis to text
    sentence = emoji.demojize(sentence)

    return sentence

In [12]:
clean_data = corpus_limpio.copy()
clean_data['texto'] = clean_data['texto'].apply(process_text)

### Eliminando las palabras que no aportan valor (stopwords)

In [46]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [47]:
nltk.download('stopwords')
print(stopwords.words('spanish'))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [48]:
stop_words = set(stopwords.words('spanish'))

In [49]:
def remove_stopwords(text):
  word_tokens = word_tokenize(text)
  no_stopwords = [word for word in word_tokens if not word in stop_words]
  return " ".join(no_stopwords)

In [ ]:
remove_stopwords('el que tiene tienda la debe atender')

In [51]:
clean_data['texto'] = clean_data['texto'].apply(remove_stopwords)

### Lematización

In [52]:
#https://spacy.io/models/es
#We'll use Spacy for Lematization
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 72.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [53]:
import spacy
import es_core_news_sm
nlp = es_core_news_sm.load()

In [54]:
def lematize(text):
    doc = nlp(text)
    lemms = []
    for token in doc:
        lemms.append(token.lemma_)
    return " ".join(lemms)

In [55]:
lematize('yo soy muy feliz con mi familia')

'yo ser mucho feliz con mi familia'

In [ ]:
clean_data['Text'] = clean_data['Text'].apply(lematize)

### Retirando elementos de puntuación y acentos (Punctuation Cleaning)

In [13]:
def cleaning_punct(text):
  token_list = gensim.utils.simple_preprocess(str(text), deacc=True)  # deacc=True remueve puntuación
  return " ".join(token_list)

In [ ]:
cleaning_punct('mi méxico querido qué fantástico')

In [14]:
clean_data['texto'] = clean_data['texto'].apply(cleaning_punct)

## 3. Inicialización de Modelo y utilidades

Selección de Modelos



In [32]:
modelsNames = {
    'bert-base-multilingual-cased' : 'BERT_b_C_mñ',
    'bert-base-multilingual-uncased': 'BERT_b_U_ml',
    'distilbert-base-uncased' : 'distilBERT_b_U',
    'roberta-base' : 'roBERTa_b',
    'dccuchile/bert-base-spanish-wwm-cased' : 'BETO_b_C'
}

modelo_seleccionado = "dccuchile/bert-base-spanish-wwm-cased" #@param ['bert-base-multilingual-cased', 'bert-base-multilingual-uncased','distilbert-base-uncased','roberta-base','dccuchile/bert-base-spanish-wwm-cased']

Definimos la conversión de etiquetas.

In [15]:
id2label = {0: "No", 1: "Si"}
label2id = {"No": 0, "Si": 1}

## Transformers

In [16]:
!pip install transformers==4.24.0
!pip install simpletransformers==0.63.11

### Cargando los modelos preentrenados

In [17]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [18]:
import logging # Import the logging module

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [19]:
clean_data2 = clean_data.copy()
clean_data2.rename(columns = {'texto':'text',rasgo:'labels'}, inplace = True)

In [20]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(clean_data2, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (170, 2)
test shape:  (43, 2)


In [33]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 1, # Usaremos una época por cuestiones de tiempo
             "overwrite_output_dir": True}

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    modelo_seleccionado,
    #'bert-base-uncased',
    num_labels=2,
    args=train_args
)

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

## Entrenamos el modelo

In [34]:
# Train the model
model.train_model(train_df)

  0%|          | 0/170 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/22 [00:00<?, ?it/s]

(22, 0.6938455186106942)

In [25]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [35]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df,f1=f1_score, acc=accuracy_score, rc=recall_score, pcs=precision_score)

  0%|          | 0/43 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [36]:
print(f" Exactitud (Accuracy): {result['acc']}")
print(f" F1-Score: {result['f1']}")
print(f" Recall: {result['rc']}")
print(f" Precisión: {result['pcs']}")

 Exactitud (Accuracy): 0.5581395348837209
 F1-Score: 0.6415094339622641
 Recall: 0.8095238095238095
 Precisión: 0.53125


### Probando el modelo

In [37]:
from sklearn.metrics import recall_score
from sklearn import metrics

In [38]:
#Recordemos nuestras clases 0 = Negativo, 1 = Positivo
clases = ['No','Sí']

In [43]:
#corpus_limpio.texto[0]
corpus_limpio.head()

,texto,presenta_apertura
0,Sigo sin entender porque la música ahorita ac...,0
1,Pues ayer estaba haciendo la tarea de metodol...,1
2,No pues yo voy a presentar este juego llamado...,0
3,Qué cómo están no pues la otra vez estaba pue...,0
4,Bueno pues a mí me gusta mucho lo que son los...,0


In [44]:
# Vamos a usar un diccionario para crear el dataset de prueba
# Frases:
datos = {
    'text': ['Sigo sin entender porque la música ahorita actualmente es tan explicita por qué todo está de la manera de fácilmente lo que se dedica alguien o algo del tipo de música todos podemos hablar de él pero okey pero ya entendimos el pedo pero la necesidad de hacerlo es otro rollo'],
    'labels': [0]
}

# Crear un DataFrame a partir del diccionario
df = pd.DataFrame(datos)

In [45]:
df.head()

,text,labels
0,Sigo sin entender porque la música ahorita act...,0


In [46]:
test = df['text'].to_numpy().tolist()
y = df['labels'].to_numpy().tolist()
print(test[0])
print(y[0])
print(len(test))
print(len(y))

Sigo sin entender porque la música ahorita actualmente es tan explicita por qué todo está de la manera de fácilmente lo que se dedica alguien o algo del tipo de música todos podemos hablar de él pero okey pero ya entendimos el pedo pero la necesidad de hacerlo es otro rollo
0
1
1


In [47]:
predictions_test = model.predict(test)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
#Accediendo la clase elegida por el modelo
print(clases[predictions_test[0][0]])

No


In [49]:
#Usando el vector de probabilidades
print(clases[np.argmax(predictions_test[1])])

No


In [50]:
test_recall = metrics.recall_score(y, predictions_test[0], average='macro')
test_f1 = metrics.f1_score(y, predictions_test[0], average='macro')
test_precision = metrics.precision_score(y, predictions_test[0], average='macro')
test_accuracy = metrics.accuracy_score(y, predictions_test[0])

In [51]:
print("Metrics results:")
print(f"Accuracy: {test_accuracy}")
print(f"F1: {test_f1}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")

Metrics results:
Accuracy: 1.0
F1: 1.0
Precision: 1.0
Recall: 1.0
